In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.patches as patches
from pydicom import dcmread, read_file
import skimage as sk
import cv2

In [10]:
#extraer info de imagenes sin hallazgos para adjuntarlas al dataset
todo= pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Todas.csv")
bboxes=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/bboxes.csv")
data=pd.read_csv("/media/deadwolf/BEC4-9B89/chext14/Data.csv")

In [6]:
NoOpacities=data[data['Finding Labels']=='No Finding']

In [8]:
paths=NoOpacities['Image Index']
labels=NoOpacities['Finding Labels']

In [9]:
df = pd.DataFrame({'id': paths, 'label': labels})

In [11]:
df.to_csv('/media/deadwolf/BEC4-9B89/chext14/NoOp.csv', index=False)